In [4]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
client = OpenAI()

In [4]:
# The assistant's 'instructions' property serves as the 'system' role's content for the chat completion api.

assistant = client.beta.assistants.create(
    name="Maths Tutor",
    instructions="You convert maths problems into Python code and then run the code to show the answer.",
    tools = [{'type':'code_interpreter'}],
    model = 'gpt-3.5-turbo'
)

In [5]:
assistant.id

'asst_HYSt1vSEPMsIFUrvgvn4bqVy'

In [7]:
# Threads currently have a time limit of 60 days after which they will be deleted automatically.
# Threads have no size limit so they are essentially a never-ending conversation. This is different from text
# completion, where we have to worry about context length and how many messages we're sending over to the 
# completion bot. Here, the assistant is smart enouth to just do a truncation of the messages in order to fit
# into its content length.
thread = client.beta.threads.create()

In [9]:
thread.id

'thread_ISTYDFjT9bBzGYhE6Bkpt2ys'

In [8]:
# The Chat GPT LLM, without the code interpreter, cannot answer this question on its own correctly. The assistant will
# use the 'code_interpreter' tool, which we allocated it to it, to unswer this question.
message_1 = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'What is 123456 times 456789?')

In [10]:
run = client.beta.threads.runs.create(
    assistant_id = assistant.id,
    thread_id = thread.id)

In [11]:
run.status

'queued'

In [13]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id)

In [14]:
run.status

'completed'

In [16]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)

In [17]:
messages

SyncCursorPage[Message](data=[Message(id='msg_LjRnPBSXqD2pLtmjWuYHUYOS', assistant_id='asst_HYSt1vSEPMsIFUrvgvn4bqVy', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='123456 multiplied by 456789 equals 56,393,342,784.'), type='text')], created_at=1718358279, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_I1hCQuwFWS0S1k5RtdAYTGdo', status=None, thread_id='thread_ISTYDFjT9bBzGYhE6Bkpt2ys'), Message(id='msg_IqBVSdBdP8skQijeQhl2WDKc', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is 123456 times 456789?'), type='text')], created_at=1718357348, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_ISTYDFjT9bBzGYhE6Bkpt2ys')], object='list', first_id='msg_LjRnPBSXqD2pLtmjWuYHUYOS', last_id='msg_IqBVSdBdP8skQijeQhl2WDKc', has_m

In [18]:
messages.data[0].content[0].text.value


'123456 multiplied by 456789 equals 56,393,342,784.'

In [19]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)

In [20]:
for step in run_steps:
    print(step.step_details)
    print('\n')

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_LjRnPBSXqD2pLtmjWuYHUYOS'), type='message_creation')


ToolCallsStepDetails(tool_calls=[CodeInterpreterToolCall(id='call_eSV1cFKijiNtSCD5C7DClf9N', code_interpreter=CodeInterpreter(input='# Calculate the product of 123456 and 456789\nresult = 123456 * 456789\nresult', outputs=[]), type='code_interpreter')], type='tool_calls')




In [21]:
# Delete the assistant:
response = client.beta.assistants.delete(assistant.id)

In [22]:
response

AssistantDeleted(id='asst_HYSt1vSEPMsIFUrvgvn4bqVy', deleted=True, object='assistant.deleted')